## This code generate the instruction spread sheet and wav files for Gorilla

### Library & Define function

In [1]:
import os
import random
import shutil
import openpyxl
import pandas as pd


In [2]:
def combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword):
    os.makedirs(new_folder, exist_ok=True)

    new_wb = openpyxl.Workbook()
    new_ws = new_wb.active

    header_written = False

    for subfolder in subfolders:
        xlsx_files = [f for f in os.listdir(os.path.join(source_folder, subfolder)) if f.endswith('.xlsx')]

        xlsx_file = next((f for f in xlsx_files if keyword in f.lower()), None)
        if not xlsx_file:
            continue

        wb = openpyxl.load_workbook(os.path.join(source_folder, subfolder, xlsx_file))
        ws = wb.active

        if not header_written:
            new_ws.append([cell.value for cell in ws[1]])
            header_written = True

        # prepare a list of odd and even row numbers
        odd_rows = [i for i in range(2, ws.max_row + 1) if i % 2 == 1]
        even_rows = [i for i in range(2, ws.max_row + 1) if i % 2 == 0]
        
        # calculate the number of rows to select from the odd and even rows list
        odd_rows_to_select = selected_rows // 2
        even_rows_to_select = selected_rows // 2
        
        # if the number of selected rows is odd, add an additional row to select from the odd rows list
        if selected_rows % 2 != 0:
            odd_rows_to_select += 1
        
        # randomly select rows from the odd and even rows list
        selected_odd_rows = random.sample(odd_rows, min(len(odd_rows), odd_rows_to_select))
        selected_even_rows = random.sample(even_rows, min(len(even_rows), even_rows_to_select))
        
        # combine both lists of selected odd and even rows
        rows_to_select = selected_odd_rows + selected_even_rows

        for selected_row in rows_to_select:
            new_ws.append([cell.value for cell in ws[selected_row]])

            audio_filename_col_index = None
            for idx, cell in enumerate(ws[1]):
                if cell.value == "Audio_Filename":
                    audio_filename_col_index = idx + 1
                    break

            if audio_filename_col_index:
                audio_filename = ws.cell(row=selected_row, column=audio_filename_col_index).value
                audio_file_path = os.path.join(source_folder, subfolder, audio_filename)
                if os.path.isfile(audio_file_path):
                    new_audio_filename = os.path.splitext(audio_filename)[0] + '_instruction' + os.path.splitext(audio_filename)[1]
                    new_audio_file_path = os.path.join(new_folder, new_audio_filename)
                    shutil.copy(audio_file_path, new_audio_file_path)

                    new_ws.cell(row=new_ws.max_row, column=audio_filename_col_index).value = new_audio_filename

    new_wb.save(os.path.join(new_folder, 'combined_rows_instr.xlsx'))



In [3]:
def create_combined_xlsx(file1, file2, new_folder, file_name, task_type):
    AP_taskType = 'task-ap-feedback'
    SD_taskType = 'task-sd-feedback'
    # Load workbooks and worksheets
    wb1 = openpyxl.load_workbook(file1)
    ws1 = wb1.active

    wb2 = openpyxl.load_workbook(file2)
    ws2 = wb2.active

    new_wb = openpyxl.Workbook()
    new_ws = new_wb.active

    # 1. Copy the header from the 1st xlsx
    header = [cell.value for cell in ws1[1]]
    new_ws.append(header)

    # 2. Find 'display', 'text', 'randomise_trials', and 'embedded' column indices
    display_col_index = header.index('display') + 1
    text_col_index = header.index('text') + 1
    randomise_trials_col_index = header.index('randomise_trials') + 1
    #embedded_col_index = header.index('embedded') + 1

    # 3. Add two lines with specific text based on input_string
    AP_intro = '<p style="font-size: 48px;">In this task, you will be presented with a sequence of complex notes/everyday sounds, then a single complex note/everyday sound. </p> <p style="font-size: 48px;"> The task is to remember the complex notes or everyday sounds in the first sequence, then decide if the following singular complex tone/everyday sound was in the first sequence or not. </p> <p style="font-size: 48px;">  After hearing both the sequence and the singular stimuli, you will be prompted to reply with "present" or "absent".<p style="font-size: 48px;"> We will start with some examples. </p>'
    AP_complexNote_intro = '<p style="font-size: 32px;"> In this next example, listen to the sequence of complex tones and respond whether the complex tone was "present" or "absent". </p>'
    SD_intro = '<p style="font-size: 48px;">In this task, you will be presented with a sequence of complex notes/everyday sounds, then another sequence of complex note/everyday sound. </p> <p style="font-size: 48px;"> The task is to remember the complex notes or everyday sounds in the first sequence, then decide if the following sequence of complex tone/everyday sound was same as the first sequence. </p> <p style="font-size: 48px;">  After hearing both sequences, you will be prompted to reply with "same" or "different".<p style="font-size: 48px;"> We will start with some examples. </p>'
    SD_complexNote_intro = '<p style="font-size: 32px;"> In this next example, listen to the sequences of complex tones and respond whether the 2nd sequence was "same" or "different". </p>'

    text_values = [AP_intro, AP_complexNote_intro] if task_type in ['AP', 'PA'] else [SD_intro, SD_complexNote_intro]
    for text_value in text_values:
        new_row = [''] * len(header)
        new_row[display_col_index - 1] = 'text'
        new_row[text_col_index - 1] = text_value
        new_ws.append(new_row)

    # 4. Copy all the nonheader lines from the first xlsx file
    for row in ws1.iter_rows(min_row=2):
        # 5.1 Copy the line
        new_row = [cell.value for cell in row]
        #new_row[embedded_col_index - 1] = input_string
        new_row[display_col_index - 1] = AP_taskType if task_type in ['AP', 'PA'] else SD_taskType
        new_ws.append(new_row)

        # # 5.2 Add one line with 'text' under the 'display' column and 'a' under the 'text' column
        # new_row = [''] * len(header)
        # new_row[display_col_index - 1] = 'text'
        # new_row[text_col_index - 1] = 'The previous example is' + row[]
        # new_ws.append(new_row)

    # 6. Add one line with 'text' under the 'display' column and 'a' under the 'text' column
    new_row = [''] * len(header)
    new_row[display_col_index - 1] = 'text' 
    new_row[text_col_index - 1] = '<p style="font-size: 32px;"> In this next example, listen to the sequence of everyday sounds and respond whether the everyday sound was "present" or "absent". </p>' if task_type in ['AP', 'PA'] else '<p style="font-size: 32px;"> In this next example, listen to the sequences of everyday sounds and respond whether the 2nd sequence was "same" or "different". </p>'
    new_ws.append(new_row)

    # 7. Copy all the nonheader lines from the second xlsx file
    for row in ws2.iter_rows(min_row=2):
        # 7.1 Copy the line
        new_row = [cell.value for cell in row]
        new_row[randomise_trials_col_index - 1] = 2
        #new_row[embedded_col_index - 1] = input_string
        new_row[display_col_index - 1] = AP_taskType if task_type in ['AP', 'PA'] else SD_taskType
        new_ws.append(new_row)

        # # 7.2 Add one line with 'text' under the 'display' column and 'a' under the 'text' column
        # new_row = [''] * len(header)
        # new_row[display_col_index - 1] = 'text'
        # new_row[text_col_index - 1] = 'a'
        # new_ws.append(new_row)

    # 8. Add one line with 'text' under the 'display' column and 'a' under the 'text' column
    new_row = [''] * len(header)
    new_row[display_col_index - 1] = 'text'
    #number of practice trials depends on the practice trial selected
    new_row[text_col_index - 1] = '<p style="font-size: 32px;"> You are done the introductory trials. </p> <p style="font-size: 32px;">  You will now be taken to a practice section with 8 trials. You must get 5/8 trials correct to continue with the rest of the experiment. </p>'
    new_ws.append(new_row)

    new_wb.save(os.path.join(new_folder, file_name))

### AP

#### complex tone VS everyday sound

In [4]:
### Parameters
selected_rows = 4  # specify the number of rows to select
keyword = 'metadata'  # specify the keyword
rootPth = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet"

###~~~~~~~~~~~~~~ AP-complex tone ~~~~~~~~~~~~~~~~
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/PA/block1_complex_tones"
new_folder =  rootPth + "/AP_instruction_section/complex_notes"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

###~~~~~~~~~~~~~ AP-everyday sound ~~~~~~~~~~~~~~~~~~~~
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/PA/block3_everyday_sounds"
new_folder =  rootPth + "/AP_instruction_section/everyday_sounds"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### Complex tone vs cat sound

In [5]:
# ### Parameters
# selected_rows = 4  # specify the number of rows to select
# keyword = 'metadata'  # specify the keyword
# rootPth = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet"

# ###~~~~~~~~~~~~~~ AP-complex tone ~~~~~~~~~~~~~~~~
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/PA/block1_complex_tones" # Don't choose same block, or randomized block value mixed
# new_folder =  rootPth + "/AP_instruction_section/complex_notes"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

# ###~~~~~~~~~~~~~ AP-cat sound ~~~~~~~~~~~~~~~~~~~~
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/PA/block2_cat_mew_sounds" # Don't choose same block, or randomized block value mixed
# new_folder =  rootPth + "/AP_instruction_section/cat_sounds"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### Instruction generation

In [6]:
### complex note and everyday sound 
# change the instruction in the code
file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/AP_instruction_section/complex_notes/combined_rows_instr.xlsx"
file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/AP_instruction_section/everyday_sounds/combined_rows_instr.xlsx"
output_file = rootPth + "/AP_instruction_section"

# ### complex note and cat sound 
# # change the instruction in the code
# file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/AP_instruction_section/complex_notes/combined_rows_instr.xlsx"
# file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/AP_instruction_section/cat_sounds/combined_rows_instr.xlsx"
# output_file = rootPth + "/AP_instruction_section"

create_combined_xlsx(file1, file2, output_file, 'AP_instruction.xlsx', 'AP')

### SD

#### complex tone vs everyday sound

In [7]:
### Parameters
selected_rows = 4  # specify the number of rows to select
keyword = 'metadata'  # specify the keyword
rootPth = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet"

###~~~~~~~~~~~~~~ SD-complex tone ~~~~~~~~~~~~~~~~
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/SD/block1_complex_tones"
new_folder = rootPth + "/SD_instruction_section/complex_notes"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

###~~~~~~~~~~~~~ SD-everyday sound ~~~~~~~~~~~~~~~~~~~~
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/SD/block2_everyday_sounds"
new_folder = rootPth + "/SD_instruction_section/everyday_sounds"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### complex tone vs cat sound

In [8]:
# ### Parameters
# selected_rows = 4  # specify the number of rows to select
# keyword = 'metadata'  # specify the keyword

# ###~~~~~~~~~~~~~~ SD-complex tone ~~~~~~~~~~~~~~~~
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/SD/block1_complex_tones" # Don't choose same block, or randomized block value mixed
# new_folder = rootPth + "/SD_instruction_section/complex_notes"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

# ###~~~~~~~~~~~~~ SD-cat sound ~~~~~~~~~~~~~~~~~~~~
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/SD/block2_cat_mew_sounds" # Don't choose same block, or randomized block value mixed
# new_folder = rootPth + "/SD_instruction_section/cat_sounds"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### SD instruction generation

In [9]:
### complex note and everyday sound 
# change the instruction in the code
file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/SD_instruction_section/complex_notes/combined_rows_instr.xlsx"
file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/SD_instruction_section/everyday_sounds/combined_rows_instr.xlsx"
output_file = rootPth + "/SD_instruction_section"

# ### complex note and cat sound
# # change the instruction in the code
# file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/SD_instruction_section/complex_notes/combined_rows_instr.xlsx"
# file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/SD_instruction_section/cat_sounds/combined_rows_instr.xlsx"
# output_file = rootPth + "/SD_instruction_section"

create_combined_xlsx(file1, file2, output_file, 'SD_instruction.xlsx', 'SD')